In [482]:
import os
from pathlib import Path
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from modules.functions import bound_logistic
from modules.plotstyle import PlotStyle
import matplotlib.colors as mcolors
import plottools.colors as clrs
import cmocean.cm as cmo
import seaborn as sns
%matplotlib qt

# init standardized plotstyle
ps = PlotStyle()

## 4AFC Analysis

- Fit logistic functions to detection proportions of all subjects to estimate their individual detection threshold
- Compare detection thresholds across the two levels of stimulus duration

First, load the data.

In [483]:
def find_nearest(array, value):
    """Like numpy-where but flexible"""
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx] 

def find_nearest_idx(array, value):
    """Like numpy-where but flexible"""
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

logistic = bound_logistic(0.25, 1) # logistic bound between chance lvl and 1

data = pd.read_csv('../data_processed/4afc_all.csv') # load 4afc data

bound_logistic ran in: 1.9073486328125e-06 sec


Now that we have the dataset, we can exclude data that may conflict with our analysis.
For the first criterion, we exclude subjects where **more than 50% of the performances where worse that chance level**. These subjects could have selected the wrong answer on purpose.

In [484]:
# This needs some attention

Now we can iterate across all subjects, fit the logistic function for both stimulus conditions and extract the respective detection thresholds on the fitted function. 

**Thresholds that are above or beyond our stimulus boundaries will be used as indicators to exclude the subject from the subsequent analysis.**

They gray lines and datapoints represent these cases.

In [485]:
names = [] # append names here
conds =  [] # append condition here
fits = [] # append fitted curve here
threshs = [] # append thresh from logistic fit here
exclude = [] # append exclude index here

for it, condition in enumerate(np.unique(data.cat)):

    # define colors for each subject
    color = iter(cm.rainbow(np.linspace(0, 1, 8)))

    # compute thresholds and exclude where threshold is out of range
    for i, name in enumerate(np.unique(data.subj)):

        # get data for this subject
        x = np.asarray(data.what[(data.cat == condition)&(data.subj == name)], dtype=float)
        y = np.asarray(data.prop[(data.cat == condition)&(data.subj == name)], dtype=float)

        # fit the logistic function to get a psychometric curve
        popt, pcov = curve_fit(logistic, x, y, method='trf', maxfev=100000)

        # make the model curve
        x_fit = np.arange(0, 0.14, 0.0001)
        y_fit = logistic(x_fit, *popt)

        # extract threshold (where detection is half of non-chance range)
        thresh = np.round(x_fit[y_fit == find_nearest(y_fit,0.625)][0], 3)
        thresh_y = y_fit[y_fit == find_nearest(y_fit,0.625)][0]

        if (thresh>0.1) or (thresh<0.01667):
            exclude.extend([1])
            names.append(name)
            fits.append(y_fit)
            conds.append(condition)
            threshs.append(thresh)
        else:
            exclude.extend([0])
            names.append(name)
            fits.append(y_fit)
            conds.append(condition)
            threshs.append(thresh)

# make a dataframe from the collected data
exclude = np.asanyarray(exclude, dtype=bool)
names = np.asarray(names, dtype=str)
fits = np.asarray(fits, dtype=float)
conds = np.asarray(conds, dtype=str)
threshs = np.asarray(threshs, dtype=float)

# make exclude for both levels
for name in np.unique(names):
    if 1 in exclude[names==name]:
        exclude[names==name] = 1

dthresh = pd.DataFrame(np.asarray([names, exclude, conds, threshs], dtype=object).T, columns=('name', 'exclude', 'cond', 'thresh'))
dthresh = dthresh.astype({"name": str, "exclude": bool, "cond": str, "thresh": float})

Now that we have the detection threshold for each person, we also want to know whether the detection thresholds differ between the two spatial frequencies. For low spatial frequencies it should be easier to detect the orientation of the grating faster.

Lets first plot a boxplot of the two groups.

In [486]:
# since where already fitting, lets also plot this
fig, ax = plt.subplots(1,3, figsize=(25*ps.cm,10*ps.cm), constrained_layout=True)

ax[0].get_shared_x_axes().join(ax[0], ax[1])
ax[0].get_shared_y_axes().join(ax[0], ax[1])

x = x_fit
x_stim = x_fit[(x_fit<0.1)&(x_fit>0.01667)]

for i, cond in enumerate(np.unique(dthresh.cond)):

    # set colors 
    color = iter(clrs.colors_tableau)
    color = iter(cmo.haline(np.linspace(0, 1, 6)))
    
    for index in dthresh.index[dthresh.cond == cond]:

        fit = fits[index]
        fit_stim = fits[index][(x_fit<0.1)&(x_fit>0.01667)]
        
        if dthresh.exclude[index] == True:
            zorder = -10
            ax[i].plot(x, fit, c='lightgray', lw=2.5, zorder=zorder)
            ax[i].plot([dthresh.thresh[index], dthresh.thresh[index]], [0.25, find_nearest(fit, 0.625)], lw=1, c='lightgray', ls='dashed', zorder=zorder)
            ax[i].plot([dthresh.thresh[index]], [find_nearest(fit, 0.625)], marker='o', c='lightgray', zorder=zorder, markersize=7.5)
            
        else: 
            c = next(color)
            ax[i].plot(x, fit, lw=2.5, c=c, ls='dashed', zorder=-1)
            ax[i].plot(x_stim, fit_stim, lw=2.5, c=c, zorder=1)
            ax[i].plot([dthresh.thresh[index], dthresh.thresh[index]], [0.25, find_nearest(fit, 0.625)], lw=1, c=c)
            ax[i].plot([dthresh.thresh[index]], [find_nearest(fit, 0.625)], marker='o', c=c, markersize=7.5)

ax[1].text(0, 0.9, '2 cpd')
ax[0].text(0, 0.9, '8 cpd')
ax[1].set_ylim(0.25, 1)
ax[0].set_ylim(0.25, 1)
ax[0].set_ylabel('prop. correct')
ax[0].set_xlabel('stim. dur. [s]')
ax[1].set_xlabel('stim. dur. [s]')
fig.legend()

/tmp/ipykernel_20632/3226321771.py:4: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax[0].get_shared_x_axes().join(ax[0], ax[1])
/tmp/ipykernel_20632/3226321771.py:5: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax[0].get_shared_y_axes().join(ax[0], ax[1])
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [488]:
# remove the invalid datasets
dthresh_valid = dthresh[dthresh["exclude"] == False]

sbp = sns.pointplot(x = 'cond', y = 'thresh', hue='name', data = dthresh_valid, palette='viridis', ax=ax[2])
ax[2].set_xlabel('stimulus')
ax[2].set_ylabel('det. thresh')

ax[2].set_xticklabels(['8 cpd', '2 cpd'])
plt.legend([],[], frameon=False)

plt.savefig('4afc_psychofit.svg')
plt.show()